In [1]:
%pip install tensorflow opencv-python mediapipe scikit-learn matplotlib pandas pyarrow wandb plotly

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/d9/64/7fdfb9386511cd6805451e012c537073a79a958a58795c4e602e538c388c/opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/c9/e4/3e645a8f87577553194a2a15383a60d61b8381cf864e903e43e4c6eb58e0/mediapipe-0.10.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/4e/53/14405a47292b59235d811a2af8634aba188ccfd1a38ef4b8042f3447d79a/scikit_learn-1.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/fc/a5/4d82be566f069d7a9a702dcdf6f9106df0e0b042e738043c0cc7ddd7e3f6/pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaini

In [2]:
import os
import math
import pandas as pd
import numpy as np
#Load parquet data into dataset_parquet for training.
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
from glob import glob
from tqdm import tqdm
from collections import Counter
import random
import time
import wandb

2024-04-24 16:46:47.695896: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-24 16:46:47.720160: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 16:46:47.720181: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 16:46:47.720195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 16:46:47.724622: I tensorflow/core/platform/cpu_feature_g

In [3]:
class CONFIG:
    # root = os.path.join("/", "kaggle", "input", "asl-signs") 
    #root = os.path.join("..","data", "ASL-ds")
    root = os.path.join(".")
    DATA_LIMIT = 2000
    BATCH_SIZE = 16
    VIDEO_LENGTH = 36
    TRAIN_VAL_SPLIT = 0.80
    WANDB_RUN = "mediapipe-asl-dataset"


In [4]:
LIPS_IDXS0 = np.array([
        61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
        291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
        78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
        95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
    ])

EYE_LEFT = np.array([33, 7, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 471, 470, 469, 472])
EYE_RIGHT = np.array([362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 476, 475, 474, 477])
# Landmark indices in original data
LEFT_HAND_IDXS0 = np.arange(468,489)
RIGHT_HAND_IDXS0 = np.arange(522,543)
LEFT_POSE_IDXS0 = np.array([502, 504, 506, 508, 510])
RIGHT_POSE_IDXS0 = np.array([503, 505, 507, 509, 511])

print("LIPS : ",len(LIPS_IDXS0))
print("EYE_LEFT : ",len(EYE_LEFT))
print("EYE_RIGHT : ",len(EYE_RIGHT))
print("LEFT_HAND : ",len(LEFT_HAND_IDXS0))
print("RIGHT_HAND : ",len(RIGHT_HAND_IDXS0))
print("LEFT_POSE : ",len(LEFT_POSE_IDXS0))
print("RIGHT_POSE : ",len(RIGHT_POSE_IDXS0))

all_selection = np.concatenate([LIPS_IDXS0, EYE_LEFT, EYE_RIGHT, LEFT_HAND_IDXS0, RIGHT_HAND_IDXS0, LEFT_POSE_IDXS0, RIGHT_POSE_IDXS0])
print(len(all_selection))

LIPS :  40
EYE_LEFT :  20
EYE_RIGHT :  20
LEFT_HAND :  21
RIGHT_HAND :  21
LEFT_POSE :  5
RIGHT_POSE :  5
132


In [6]:
#this code sorts out a parquet files and rearrange the order to pose,face, left-hand, right-hand
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objs as go
import plotly.io as pio

ids = None

order_global = {"pose" : 10000, "face" : 1000, "left_hand" : 100, "right_hand" : 10}

def visualize_keypoints(frames : np.ndarray, point_size : int):
    if len(frames.shape) == 1:
        frames = np.array([frames])
    
    for frame in frames:
        frame = frame.reshape(-1, 3)
        sizes = point_size * np.ones(frame.shape[0])

        fig = go.Figure(data=go.Scatter(x=frame[:,0], y=2.5 - frame[:,1], mode='markers',
                                        marker=dict(
                                            size=sizes
                                            )))

    # Customize the layout
    fig.update_layout(title='visualization of human keypoints',
                        xaxis_title='',
                        yaxis_title='',
                        width=1000,
                        height=1600)

    fig.update_xaxes(range=[-0.2, 1.4])  # Set x-axis range from 0 to 6
    fig.update_yaxes(range=[0, 2.5])  # Set y-axis range from 10 to 20

    # Show the plot
    fig.show()


def process_parquet(ds, idxes = None):
    ret = []
    frames_unique = sorted(np.unique(ds["frame"]))
    for i,frame in enumerate(frames_unique):
        frame_ds = ds[ds['frame'] == frame]
        
        order = []
        for el in frame_ds["row_id"]:
            _frame, part, keypoint = el.split("-")
            order.append(order_global[part] - int(keypoint))

        order = np.array(order)
        frame_ds.iloc[:, 1] = order
        frame_ds = frame_ds.sort_values(by="row_id", ascending=False)
    
        vals = np.array(frame_ds[["x", "y", "z"]])
        if idxes is not None:
            vals = vals[idxes]
    
        vals = vals.flatten()

        ret.append(vals)
        
    return np.array(ret)


def process_parquet2(ds, idxes = None):
    ret = []    
    frame_size = 543
    it = len(ds) // frame_size
    assert it == len(ds) / frame_size
    
    for i in range(it):
        vals = ds.iloc[ i * frame_size : (i + 1 ) * frame_size ]        
        
        if idxes is not None:          
            vals = ds.iloc[idxes]
                        
        ret.append(np.array(vals[["x","y", "z"]]).flatten())
        
    return np.array(np.array(ret))

In [8]:
from sklearn.model_selection import train_test_split

#custom class to load data from Parquet files for training ML models.
class ParquetDatasetCached(keras.utils.Sequence):
    def __init__(self, dataset_folder, csv_file : str, batch_size=CONFIG.BATCH_SIZE, 
                 data_limit :int= CONFIG.DATA_LIMIT, check_if_file_exists = True, 
                 preprocessing_func=None, frame_length :int = CONFIG.VIDEO_LENGTH,
                 split : str = "train", train_val_split : float = CONFIG.TRAIN_VAL_SPLIT,
                 sort_by_counts : bool = True, **kwargs
                ):
        super().__init__(**kwargs)
        #taking keras sequence for .fit(), .evaluate(), .predict() methods
        #load csv - it has the path to parquet file, and another to store label
        self.csv_path = csv_file
        self.root_folder = dataset_folder
        self.batch_size = batch_size
        #optional pre-processing function to the parquet files.
        self.preprocessing_func = preprocessing_func
        
        self.csv_data = pd.read_csv(self.csv_path)
        
        self.all_files = []
        self.not_exists = []
        self.frame_length = frame_length

        
        for path, label in tqdm(list(zip(self.csv_data["path"], self.csv_data["sign"]))):
            prop_path = os.path.join(self.root_folder, path)
            
            if check_if_file_exists:
                if os.path.exists(prop_path):
                    self.all_files.append((prop_path, label))
                else:
                    self.not_exists.append(prop_path)
            else:
                self.all_files.append((prop_path, label))
                
                    
        self.all_files = np.array(self.all_files)
        self.unique_labels = np.unique(self.all_files[:, 1])
        self.label_2_id = { key : i for i, key in enumerate(self.unique_labels)}
    
        # sort the values by popularity
        if sort_by_counts:
            cnt = Counter(self.all_files[:, 1])
            vals = []
            
            for i,row in enumerate(self.all_files):
                vals.append((int(1e6 * cnt[row[1]] + self.label_2_id [row[1]]),i))
            
            vals = np.array(sorted(vals)[::-1])
            self.all_files = self.all_files[vals[:,1]]

        
        if data_limit < 0:
            train_ds, val_ds = train_test_split(self.all_files, train_size=train_val_split, random_state=42)
        else:
            train_ds, val_ds = train_test_split(self.all_files[:data_limit], train_size=train_val_split, random_state=42)
            self.unique_labels = np.unique(self.all_files[:data_limit, 1])
            self.label_2_id = { key : i for i, key in enumerate(self.unique_labels)}
            
        if split.lower() == "train":
            self.dataset = train_ds
            
        elif split.lower() == "val":
            self.dataset = val_ds 
            
        else:
            raise Exception("please specify split to be either train or val")
            
        self.cache_data()
                   

    def cache_data(self):
        self.cached_X, self.cached_Y = [], []
        
        pb = tqdm(range(len(self.dataset)), desc="Cacheing")

        for i in pb:
            
            path, label = self.dataset[i]
            df = pd.read_parquet(path)
                        
            one_hot_encoded_label = np.zeros(len(self.unique_labels))
            one_hot_encoded_label[self.label_2_id[label]] = 1

            self.cached_X.append(process_parquet2(df)) 
            self.cached_Y.append(one_hot_encoded_label)        

    def __len__(self):
        # Assuming each Parquet file should be one batch; adjust if necessary
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.preprocessing_func(self.cached_X[idx]), self.cached_Y[idx]                
        
    def on_epoch_end(self):
        # Shuffle files for the next epoch
        np.random.shuffle(self.dataset)


def preprocess(frames):
    current_length, num_features = frames.shape

    if current_length >= CONFIG.VIDEO_LENGTH:
            # TODO: a better than uniform value ? Could place gaussian in the middle
            random_start = random.randint(0, current_length - CONFIG.VIDEO_LENGTH)
            return np.nan_to_num(frames[random_start : (random_start + CONFIG.VIDEO_LENGTH)])
        
    # padd the video to contain zeros 
    return np.concatenate([np.nan_to_num(frames), np.zeros((CONFIG.VIDEO_LENGTH - current_length, num_features))], axis=0)

    
# Usage example
parquet_folder_path = CONFIG.root
train_dataset_parquet = ParquetDatasetCached(parquet_folder_path, csv_file = os.path.join(CONFIG.root, "train.csv"), 
                                 batch_size=CONFIG.BATCH_SIZE, data_limit=CONFIG.DATA_LIMIT,
                                 preprocessing_func=preprocess,
                                check_if_file_exists = True,
                                split="train")

val_dataset_parquet = ParquetDatasetCached(parquet_folder_path, csv_file = os.path.join(CONFIG.root, "train.csv"), 
                                 batch_size=CONFIG.BATCH_SIZE, data_limit=CONFIG.DATA_LIMIT,
                                 preprocessing_func=preprocess,
                                 check_if_file_exists= True,
                                 split="val")

print(f"cardinality of train : {len(train_dataset_parquet)}, cardinality of validation : {len(val_dataset_parquet)}")

Cacheing: 100%|██████████| 400/400 [00:05<00:00, 75.97it/s]

cardinality of train : 1600, cardinality of validation : 400


In [9]:
X_shape = train_dataset_parquet[0][0].shape
Y_shape = train_dataset_parquet[0][1].shape
print(f"X_shape = {X_shape}, Y_shape = {Y_shape}")

X_shape = (36, 1629), Y_shape = (5,)


In [10]:
print(train_dataset_parquet.label_2_id)

{'donkey': 0, 'listen': 1, 'look': 2, 'mouse': 3, 'shhh': 4}


In [11]:
def dataset_train_generator():
    # Instantiate your existing dataset loader

    for i in range(len(train_dataset_parquet)):
        X_batch, Y_batch = train_dataset_parquet[i]
        yield X_batch, Y_batch
        
def dataset_val_generator():
    # Instantiate your existing dataset loader

    for i in range(len(val_dataset_parquet)):
        X_batch, Y_batch = val_dataset_parquet[i]
        yield X_batch, Y_batch        


train_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_train_generator(),
    output_types=(tf.float32, tf.float32),  # Adjust types based on your actual data
    output_shapes=(X_shape, Y_shape)
).prefetch(tf.data.AUTOTUNE).batch(CONFIG.BATCH_SIZE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_val_generator(),
    output_types=(tf.float32, tf.float32),  # Adjust types based on your actual data
    output_shapes=(X_shape, Y_shape)
).prefetch(tf.data.AUTOTUNE).batch(CONFIG.BATCH_SIZE)

2024-04-24 17:10:25.671640: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22462 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


In [12]:
start = time.time()
isnans =False

f = True
labels_batches = []
for el in tqdm(train_dataset):
    if f:
        print(el[0].shape, el[1].shape)
        f = False
    labels_batches.append(el[1])
        
    isnans |= np.any(np.isnan(el[0]))
    if isnans:
        print("FOUND NAN!")
        break


print(f"Iterating through dataset took : {round( time.time() - start , 4)}s")

29it [00:00, 154.56it/s]

(16, 36, 1629) (16, 5)


100it [00:00, 200.17it/s]

Iterating through dataset took : 0.5031s


# Train experiment

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
from wandb.keras import WandbMetricsLogger
from keras.callbacks import Callback
import tensorflow as tf
import wandb


class CosineAnnealingLearningRateScheduler(Callback):
    def __init__(self, max_lr, min_lr, T_max):
        super(CosineAnnealingLearningRateScheduler, self).__init__()
        self.max_lr = max_lr  # Maximum learning rate (i.e., start learning rate)
        self.min_lr = min_lr  # Minimum learning rate
        self.T_max = T_max    # Specifies the number of epochs per cycle
        self.t = 0            # Current epoch

    def on_epoch_begin(self, epoch, logs=None):
        self.t += 1
        cos = np.cos(np.pi * (self.t % self.T_max) / self.T_max)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + cos)

        keras.backend.set_value(self.model.optimizer.lr, lr)

def keras_train(model, filepath : str, max_lr = 1e-4, min_lr = 2.5e-5, T_max=101, epochs=200, run_name="",
                mediapipe_features = "all", USE_WANDB=True,
               verbose="auto"): 
    
    
    checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                                 monitor="val_categorical_accuracy",
                                                 verbose=0,
                                                 save_best_only=True,
                                                 mode="max",
                                                 save_freq="epoch")
    
    cosine_annealer = CosineAnnealingLearningRateScheduler(max_lr=max_lr,
                                                           min_lr=min_lr,
                                                           T_max=T_max)
    
    #Adam Optimizer - fixed learning rate.
    adam_optimizer = tf.keras.optimizers.Adam(learning_rate=max_lr, clipnorm=1.)

    model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    
    callbacks  = [checkpoint, cosine_annealer]
    
    if USE_WANDB:
        wandb.init(project=CONFIG.WANDB_RUN,
                        name=run_name,
                        notes="Model summary : \n" + str(model.summary()),
                        config={"max_lr" : max_lr, 
                                "min_lr" : 5e-5, 
                                "scheduler" : "cosineAnnealer", 
                                "epochs" : epochs, 
                                "T_max" : T_max, 
                                "train_size" : len(train_dataset_parquet.dataset),
                                "val_size" : len(val_dataset_parquet.dataset),
                                "unique_classes" : len(train_dataset_parquet.unique_labels), 
                                "video_length" : CONFIG.VIDEO_LENGTH,
                                "features" : mediapipe_features
                                })
        callbacks.append(WandbMetricsLogger())


    
    history = model.fit(train_dataset, epochs=epochs, validation_data = val_dataset, callbacks=callbacks, verbose=verbose)
    
    if USE_WANDB:      
        wandb.finish()
    
    return history

## LSTM

In [ ]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(16, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM16-LR-Dense256-fs=36-cls=5.tf"),
            run_name="AK-LSTM16-LR-Dense256-fs=36-cls=",
            USE_WANDB=True)

2024-04-24 17:10:36.148165: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 16)                105344    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16)                0         
                                                                 
 dense (Dense)               (None, 256)               4352      
                                                                 
 dense_1 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 110981 (433.52 KB)
Trainable params: 110981 (433.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

100/100 [==============================] - 5s 48ms/step - loss: 0.6758 - categorical_accuracy: 0.7325 - val_loss: 0.6619 - val_categorical_accuracy: 0.7225
Epoch 128/200
100/100 [==============================] - 1s 6ms/step - loss: 0.6512 - categorical_accuracy: 0.7450 - val_loss: 0.7779 - val_categorical_accuracy: 0.6725
Epoch 129/200
100/100 [==============================] - 1s 6ms/step - loss: 0.7258 - categorical_accuracy: 0.7119 - val_loss: 0.6725 - val_categorical_accuracy: 0.7125
Epoch 130/200
100/100 [==============================] - 1s 6ms/step - loss: 0.6662 - categorical_accuracy: 0.7481 - val_loss: 0.7657 - val_categorical_accuracy: 0.6800
Epoch 131/200
100/100 [==============================] - 1s 6ms/step - loss: 0.6569 - categorical_accuracy: 0.7444 - val_loss: 0.6912 - val_categorical_accuracy: 0.7025
Epoch 132/200
100/100 [==============================] - 1s 6ms/step - loss: 0.6776 - categorical_accuracy: 0.7394 - val_loss: 0.7971 - val_categorical_accuracy: 0.6550

INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 5s 48ms/step - loss: 0.6265 - categorical_accuracy: 0.7588 - val_loss: 0.6494 - val_categorical_accuracy: 0.7550
Epoch 157/200
100/100 [==============================] - 1s 6ms/step - loss: 0.6079 - categorical_accuracy: 0.7644 - val_loss: 0.7183 - val_categorical_accuracy: 0.6800
Epoch 158/200
100/100 [==============================] - 1s 6ms/step - loss: 0.6134 - categorical_accuracy: 0.7681 - val_loss: 0.6749 - val_categorical_accuracy: 0.7000
Epoch 159/200
100/100 [==============================] - 1s 6ms/step - loss: 0.6123 - categorical_accuracy: 0.7706 - val_loss: 0.7318 - val_categorical_accuracy: 0.6825
Epoch 160/200
100/100 [==============================] - 1s 6ms/step - loss: 0.6014 - categorical_accuracy: 0.7719 - val_loss: 0.6844 - val_categorical_accuracy: 0.6950
Epoch 161/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5769 - categorical_accuracy: 0.7881 - val_loss: 0.6929 - val_categorical_accuracy: 0.7200

INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 5s 48ms/step - loss: 0.6024 - categorical_accuracy: 0.7694 - val_loss: 0.6092 - val_categorical_accuracy: 0.7625
Epoch 167/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5719 - categorical_accuracy: 0.7869 - val_loss: 0.6394 - val_categorical_accuracy: 0.7225
Epoch 168/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5706 - categorical_accuracy: 0.7825 - val_loss: 0.6631 - val_categorical_accuracy: 0.7175
Epoch 169/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5800 - categorical_accuracy: 0.7744 - val_loss: 0.6421 - val_categorical_accuracy: 0.7425
Epoch 170/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5543 - categorical_accuracy: 0.7887 - val_loss: 0.6521 - val_categorical_accuracy: 0.7200
Epoch 171/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5694 - categorical_accuracy: 0.7850 - val_loss: 0.6633 - val_categorical_accuracy: 0.7400

In [ ]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(32, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM32-LR-Dense256-fs=36-cls=5.tf"),
            run_name="AK-LSTM32-LR-Dense256-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(64, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM64-LR-Dense256-fs=36-cls=5.tf"),
            run_name="AK-LSTM64-LR-Dense256-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(128, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM128-LR-Dense256-fs=36-cls=5.tf"),
            run_name="AK-LSTM128-LR-Dense256-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM256-LR-Dense128-gelu-fs=36-cls=5.tf"),
            run_name="AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(512, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM512-LR-Dense128-gelu-fs=36-cls=5.tf"),
            run_name="AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3",
            USE_WANDB=True)

## GRU

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(16, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU16-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU16-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(32, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU32-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU32-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(64, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU64-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU64-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(128, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU128-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU128-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(256, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU256-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU256-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(512, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU512-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU512-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

## Bidirectional LSTM

In [21]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(16, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM16-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BLSTM16-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 5s 8ms/step - loss: 1.0840 - categorical_accuracy: 0.4105

2024-04-22 12:38:18.914113: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 12:38:18.914192: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 18252922663707797810
2024-04-22 12:38:18.914211: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650
2024-04-22 12:38:18.914267: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f2044da4f50>, 139805954698224), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f2044da4f50>, 139805954698224), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1ff447c550>, 139775227457840), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1ff591fd90>, 139776573174080), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM16-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM16-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 22s 415ms/step - loss: 0.1709 - categorical_accuracy: 0.9482 - val_loss: 0.2742 - val_categorical_accuracy: 0.9267
Epoch 195/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1977 - categorical_accuracy: 0.9435 - val_loss: 0.2974 - val_categorical_accuracy: 0.9200
Epoch 196/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1757 - categorical_accuracy: 0.9518 - val_loss: 0.3225 - val_categorical_accuracy: 0.8867
Epoch 197/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2120 - categorical_accuracy: 0.9365 - val_loss: 0.3382 - val_categorical_accuracy: 0.8800
Epoch 198/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1648 - categorical_accuracy: 0.9553 - val_loss: 0.3465 - val_categorical_accuracy: 0.9000
Epoch 199/200
54/54 [==============================] - 1s 9ms/step - loss: 0.1867 - categorical_accuracy: 0.9553 - val_loss: 0.5068 - val_categorical_accuracy: 0.8733
Epoch

epoch/categorical_accuracy,▁▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇█████▇█████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇███
epoch/val_loss,█▇▆▃▄▄▄▃▄▂▂▂▃▃▃▂▂▂▃▃▃▃▃▃▂▁▂▂▃▂▂▃▂▂▂▂▂▂▁▁
epoch/categorical_accuracy,0.94824
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.18871
epoch/val_categorical_accuracy,0.91333


In [22]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(32, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM32-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BLSTM32-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 5s 9ms/step - loss: 1.0742 - categorical_accuracy: 0.4216

2024-04-22 12:47:50.536797: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14635252564011947125
2024-04-22 12:47:50.536844: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650
2024-04-22 12:47:50.536871: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbe733650>, 139775238372832), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbe733650>, 139775238372832), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbe7cba10>, 139774325937456), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbdb11d50>, 139774341087840), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM32-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM32-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 21s 393ms/step - loss: 0.2835 - categorical_accuracy: 0.9129 - val_loss: 0.3580 - val_categorical_accuracy: 0.9200
Epoch 76/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2899 - categorical_accuracy: 0.9153 - val_loss: 0.4352 - val_categorical_accuracy: 0.8533
Epoch 77/200
54/54 [==============================] - 1s 11ms/step - loss: 0.3016 - categorical_accuracy: 0.9224 - val_loss: 0.4281 - val_categorical_accuracy: 0.8667
Epoch 78/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2656 - categorical_accuracy: 0.9282 - val_loss: 0.4697 - val_categorical_accuracy: 0.8600
Epoch 79/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2553 - categorical_accuracy: 0.9294 - val_loss: 0.4806 - val_categorical_accuracy: 0.8800
Epoch 80/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2490 - categorical_accuracy: 0.9294 - val_loss: 0.4406 - val_categorical_accuracy: 0.8667
Epoch 81/

epoch/categorical_accuracy,▁▅▆▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▄▄▆▅▄▆▆▆▆▇▇▇▆▇▆█▇█▇▇█▆▇▇█▆▆▇▇▇▇██▇▇███
epoch/val_loss,█▅▆▅▄▅▆▂▄▃▄▂▃▂▃▃▂▁▂▂▃▂▂▄▂▃▃▄▄▂▃▂▂▂▁▃▂▁▁▁
epoch/categorical_accuracy,0.95412
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.16424
epoch/val_categorical_accuracy,0.9


In [23]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(64, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM64-LR-Dense256-gelu-cls=3.tf"),
            run_name="AK-BLSTM64-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 5s 9ms/step - loss: 1.1174 - categorical_accuracy: 0.4645

2024-04-22 12:56:01.007349: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8866511902849424037
2024-04-22 12:56:01.007443: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6419013273699342798
2024-04-22 12:56:02.021207: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 12:56:02.021303: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9122064850495178467
2024-04-22 12:56:02.021348: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9df5f310>, 139776572418720), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9df5f310>, 139776572418720), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9d6a9550>, 139773748536496), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbc7f3ad0>, 139773393444400), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM64-LR-Dense256-gelu-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM64-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 23s 429ms/step - loss: 0.2175 - categorical_accuracy: 0.9471 - val_loss: 0.3553 - val_categorical_accuracy: 0.9267
Epoch 132/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2547 - categorical_accuracy: 0.9329 - val_loss: 0.4296 - val_categorical_accuracy: 0.8733
Epoch 133/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2217 - categorical_accuracy: 0.9494 - val_loss: 0.6109 - val_categorical_accuracy: 0.8667
Epoch 134/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2728 - categorical_accuracy: 0.9259 - val_loss: 0.4151 - val_categorical_accuracy: 0.8667
Epoch 135/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2595 - categorical_accuracy: 0.9282 - val_loss: 0.4296 - val_categorical_accuracy: 0.8933
Epoch 136/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2485 - categorical_accuracy: 0.9424 - val_loss: 0.3412 - val_categorical_accuracy: 0.9067
Epoch

epoch/categorical_accuracy,▁▄▅▆▆▆▇▆▇▇▇▇▇▇▇█████▇▇▇▇█▇▇▇█▇██████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▄▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▃▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▄▆▆▄▄▅▅▅▇▇▇▇▇▇█▇▇▇▇▇█▆▆█▇▇▇▆▇██▇▇▇██▇██
epoch/val_loss,█▅▄▄█▇▄▆▅▂▃▂▂▃▂▁▂▂▂▃▂▃▄▂▁▂▂▁▃▂▁▁▂▁▁▁▂▁▂▂
epoch/categorical_accuracy,0.95765
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.18169
epoch/val_categorical_accuracy,0.89333


In [24]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(128, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM128-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BLSTM128-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     50/Unknown - 5s 9ms/step - loss: 1.1848 - categorical_accuracy: 0.4575

2024-04-22 13:03:09.148544: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2737983808172683809
2024-04-22 13:03:09.148639: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8866511902849424037
2024-04-22 13:03:09.148669: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7134357954231986336
2024-04-22 13:03:09.148747: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6419013273699342798
2024-04-22 13:03:10.147553: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1570638488140254319
2024-04-22 13:03:10.147610: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 13:03:10.147643: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv it

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9e53ab90>, 139773772335120), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9e53ab90>, 139773772335120), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(128, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f6018c8d0>, 139775234956720), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9df37c50>, 139774320689760), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM128-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM128-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 23s 433ms/step - loss: 0.1622 - categorical_accuracy: 0.9753 - val_loss: 0.3317 - val_categorical_accuracy: 0.9267
Epoch 196/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1656 - categorical_accuracy: 0.9706 - val_loss: 0.3861 - val_categorical_accuracy: 0.9000
Epoch 197/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1566 - categorical_accuracy: 0.9741 - val_loss: 0.4553 - val_categorical_accuracy: 0.8733
Epoch 198/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1637 - categorical_accuracy: 0.9671 - val_loss: 0.4347 - val_categorical_accuracy: 0.9067
Epoch 199/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1584 - categorical_accuracy: 0.9682 - val_loss: 0.3544 - val_categorical_accuracy: 0.9000
Epoch 200/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1665 - categorical_accuracy: 0.9671 - val_loss: 0.4274 - val_categorical_accuracy: 0.8933


epoch/categorical_accuracy,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇█████▇▇▇▇▇▇▇▇▇██▇████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▅▅▇▆▇█▇▇▄▆▇██▇█▇▇▇▇▆▇▆▇▇▆▇▇▇▇█▇▇▇▇█████
epoch/val_loss,█▅▅▃▄▃▂▂▃▆▄▃▂▁▂▂▃▂▂▃▆▂▃▄▂▄▂▃▁▂▁▂▂▃▂▁▁▂▂▂
epoch/categorical_accuracy,0.96706
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.16646
epoch/val_categorical_accuracy,0.89333


In [25]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM256-LR-Dense256-gelu-cls="),
            run_name="AK-BLSTM256-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     49/Unknown - 5s 9ms/step - loss: 1.2938 - categorical_accuracy: 0.4554

2024-04-22 13:10:18.669765: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3c7593d0>, 139774286764992), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3c7593d0>, 139774286764992), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(256, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3c78a490>, 139772389045552), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f859cc750>, 139772734880816), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM256-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM256-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 22s 406ms/step - loss: 0.2760 - categorical_accuracy: 0.9447 - val_loss: 0.4081 - val_categorical_accuracy: 0.9200
Epoch 88/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2635 - categorical_accuracy: 0.9565 - val_loss: 0.5574 - val_categorical_accuracy: 0.8600
Epoch 89/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2437 - categorical_accuracy: 0.9647 - val_loss: 0.4293 - val_categorical_accuracy: 0.9133
Epoch 90/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2807 - categorical_accuracy: 0.9459 - val_loss: 0.5564 - val_categorical_accuracy: 0.8800
Epoch 91/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2847 - categorical_accuracy: 0.9471 - val_loss: 0.4785 - val_categorical_accuracy: 0.8667
Epoch 92/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2562 - categorical_accuracy: 0.9600 - val_loss: 0.4919 - val_categorical_accuracy: 0.8867
Epoch 93/

epoch/categorical_accuracy,▁▄▅▅▆▆▇▇▇▇▇▇██▇█████▇▇▇▇▇█▇▇████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▅▃▄▄▇▇▅▇▆▃▇▆▆███▇▇▇▇▄▇▄▆▆▇█▇▅▅▆▇▆▇▇▇█▇█
epoch/val_loss,█▅▇▆▅▃▄▅▂▄▇▃▃▃▂▂▃▃▃▂▁▇▂▆▃▃▃▁▂▆▅▃▂▃▂▃▂▂▁▂
epoch/categorical_accuracy,0.97529
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.15214
epoch/val_categorical_accuracy,0.89333


## Bidirectional GRU

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(16, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU16-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU16-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(32, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU32-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU32-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(64, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU64-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU64-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(128, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU128-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU128-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU256-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU256-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)